In [47]:
import sys
import os
import numpy as np
from datetime import datetime
import glob
#Get the path of the current file
current_path = os.getcwd()
# Add the evoman_framework folder to sys.path
sys.path.append(os.path.join(current_path, "evoman"))
# Now you can import contents from the evoman environment
from evoman.environment import Environment
from evoman.controller import Controller
# Initialize environment

# Set headless mode to improve running speed
headless = True
if headless:
    os.environ["SDL_VIDEODRIVER"] = "dummy"

experiment_name = "multi_island_optimization"
test_log_path = 'multi_island_optimization/test_logs/'
module_path = 'multi_island_optimization/models/' 
if not os.path.exists(experiment_name):
    os.makedirs(experiment_name)

if not os.path.exists(test_log_path):
    os.makedirs(test_log_path)

if not os.path.exists(module_path):
    os.makedirs(module_path)
    
# Define Sigmoid activation function
def sigmoid_activation(x):
    return 1.0 / (1.0 + np.exp(-x))
# Custom controller with a simple neural network structure
class player_controller(Controller):
    def __init__(self, _n_hidden):
        self.n_hidden = [_n_hidden]  # Number of neurons in the hidden layer

    def set(self, controller, n_inputs):
        if self.n_hidden[0] > 0:
            # Extract weights and biases from controller parameters to build the neural network
            self.bias1 = controller[: self.n_hidden[0]].reshape(1, self.n_hidden[0])
            weights1_slice = n_inputs * self.n_hidden[0] + self.n_hidden[0]
            self.weights1 = controller[self.n_hidden[0] : weights1_slice].reshape(
                (n_inputs, self.n_hidden[0])
            )
            self.bias2 = controller[weights1_slice : weights1_slice + 5].reshape(1, 5)
            self.weights2 = controller[weights1_slice + 5 :].reshape(
                (self.n_hidden[0], 5)
            )

    def control(self, inputs, controller):
        # Normalize inputs
        inputs = (inputs - min(inputs)) / float((max(inputs) - min(inputs)))
        if self.n_hidden[0] > 0:
            output1 = sigmoid_activation(inputs.dot(self.weights1) + self.bias1)
            output = sigmoid_activation(output1.dot(self.weights2) + self.bias2)[0]
        else:
            bias = controller[:5].reshape(1, 5)
            weights = controller[5:].reshape((len(inputs), 5))
            output = sigmoid_activation(inputs.dot(weights) + bias)[0]

        # Determine player actions based on output
        left = 1 if output[0] > 0.5 else 0
        right = 1 if output[1] > 0.5 else 0
        jump = 1 if output[2] > 0.5 else 0
        shoot = 1 if output[3] > 0.5 else 0
        release = 1 if output[4] > 0.5 else 0

        return [left, right, jump, shoot, release]


# Initialize controller
player_controller_instance = player_controller(10)  # Example with 10 hidden neurons



def initialize_islands(enemylist):
    env.enemies = enemylist
    env.multiplemode="yes"
    islands = []
    for _ in range(n_islands):
        #生成一个二维数组pop，维度分别为个体数目，以及特征数目
        pop = np.random.uniform(dom_l, dom_u, (npop // n_islands, n_vars))
        fit_pop = evaluate(pop)
        islands.append((pop, fit_pop))
        #一个元组列表
    return islands


def migrate(islands):
    num_migrants = int(migration_rate * (npop // n_islands))
    for i in range(n_islands):
        # Select migrants from island i
        migrants = islands[i][0][:num_migrants]
        # Migrate to the next island (circular migration)
        target_island = (i + 1) % n_islands
        islands[target_island][0][-num_migrants:] = migrants
    return islands


# Uniform crossover
def uniform_crossover(p1, p2):
    offspring = np.zeros_like(p1)
    for i in range(len(p1)):
        if np.random.rand() < 0.5:
            offspring[i] = p1[i]
        else:
            offspring[i] = p2[i]
    return offspring


# Fixed mutation
def mutation_operation(offspring):
    for i in range(len(offspring)):
        if np.random.uniform(0, 1) <= mutation:
            offspring[i] += np.random.normal(0, 1)
    return offspring


# Elitism selection
def elitism(pop, fit_pop, elite_size):
    elite_indices = np.argsort(fit_pop)[-elite_size:]
    elite_pop = pop[elite_indices]
    elite_fitness = fit_pop[elite_indices]
    return elite_pop, elite_fitness


# Crossover and mutation operations
def crossover_and_mutation(pop):
    total_offspring = np.zeros((0, n_vars))
    for p in range(0, pop.shape[0], 2):
        p1 = pop[np.random.randint(0, pop.shape[0])]
        p2 = pop[np.random.randint(0, pop.shape[0])]

        # Uniform crossover
        offspring = uniform_crossover(p1, p2)

        # Fixed mutation operation
        offspring = mutation_operation(offspring)

        # Fitness constraint
        offspring = np.clip(offspring, dom_l, dom_u)
        total_offspring = np.vstack((total_offspring, offspring))

    return total_offspring 

def evaluate(x):
    #把每一列的权重进行测试
    return np.array([simulation(env, ind) for ind in x])

def simulation(env, x):
    env.player_controller.set(x, n_inputs)  # Set controller parameters
    f, _, _, _ = env.play(pcont=x)
    return f


def evolve_islands(islands,enemylist):
    for i in range(n_islands):
        
        env.enemies = [enemylist[i]]
        env.multiplemode="no"
        pop, fit_pop = islands[i]
        elite_size = int(elitism_rate * len(pop))
        elite_pop, elite_fitness = elitism(pop, fit_pop, elite_size)

        # Crossover and mutation
        offspring = crossover_and_mutation(pop)
        
        env.enemies = enemylist
        env.multiplemode="yes"
        fit_offspring = evaluate(offspring)

        # Combine elites and offspring
        pop = np.vstack((elite_pop, offspring[: len(pop) - elite_size]))
        fit_pop = np.hstack((elite_fitness, fit_offspring[: len(pop) - elite_size]))

        # Update the island's population and fitness
        islands[i] = (pop, fit_pop)
    return islands

# Run evolution process
def run_evolution(enemylist):
    current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"{experiment_name}/test_logs/MI_log_{current_time}.txt"

    with open(filename, "a") as file:
        file.write(
            f"npop{npop}_gens{gens}_"
            f"enemy{enemylist}_mut{mutation:.2f}_elitism{elitism_rate:.2f}_{current_time}\n"
        )
        file.write("gen best mean std\n")

        islands = initialize_islands(enemylist)

        for generation in range(gens):
            
            
            # Evolve each island
            islands = evolve_islands(islands,enemylist)

            
            
            
            # Perform migration every migration_interval generations
            if generation % migration_interval == 0 and generation > 0:
                islands = migrate(islands)

            
            # Record results (best, mean, std across all islands)
            all_fitness = np.hstack([fit_pop for _, fit_pop in islands])
            best = np.max(all_fitness)
            mean = np.mean(all_fitness)
            std = np.std(all_fitness)

            result = f"{generation} {best:.6f} {mean:.6f} {std:.6f}"
            print(result)
            file.write(result + "\n")

        # Find the best solution across all islands
        all_pop = np.vstack([pop for pop, _ in islands])
        all_fit_pop = np.hstack([fit_pop for _, fit_pop in islands])
        save_final_solution(all_pop, all_fit_pop, experiment_name, env.enemies, npop, gens, mutation, elitism_rate)

def save_final_solution(
    pop, fit_pop, experiment_name, enemies, npop, gens, mutation, elitism_rate
):
    best_index = np.argmax(fit_pop)
    best_solution = pop[best_index]
    best_fitness = fit_pop[best_index]

    # Get current timestamp
    current_time = datetime.now().strftime("%Y%m%d_%H%M%S")

    # Filename uses the best fitness
    filename = (
        f"{experiment_name}/models/MIModel_fitness{best_fitness:.6f}_npop{npop}_gens{gens}_"
        f"enemy{enemies}_mut{mutation:.2f}_elitism{elitism_rate:.2f}_{current_time}.txt"
    )

    # Save the best individual's controller weights and biases
    np.savetxt(filename, best_solution)

    # Print information about the saved file
    print(f"Best solution saved as {filename} with fitness: {best_fitness:.6f}")
 
    
    
    


In [49]:
env = Environment(
    experiment_name=experiment_name,
    #multiplemode="yes",
    playermode="ai",
    player_controller=player_controller_instance,
    enemymode="static",
    level=2,
    speed="fastest",
    visuals=False,
)
#set your enemy here
enemylist = [1,2,3,4]

# Genetic algorithm parameters
n_inputs = 20  # Number of inputs to the controller
n_vars = (
    n_inputs * 10 + 10 + 10 * 5 + 5
)  # Total number of weights and biases in the controller
dom_u = 1
dom_l = -1
npop = 100  # Population size
gens = 500  # Total generations
mutation = 0.2  # Mutation rate
elitism_rate = 0.2  # Elitism rate
migration_interval = 5  # Every 10 generations
migration_rate = 0.4  # 10% of the population migrates
n_islands = 4

run_evolution(enemylist)


MESSAGE: Pygame initialized for simulation.
0 25.195968 -2.973995 6.766260
1 25.195968 -1.742582 8.401285
2 25.195968 0.009907 10.081314
3 25.195968 -1.197710 10.077570
4 25.195968 3.719362 11.893262
5 26.451599 2.629273 12.266851
6 26.451599 2.269425 11.571458
7 26.451599 2.748089 11.003642
8 26.451599 4.605116 11.318654
9 26.451599 5.932654 11.784363
10 26.451599 7.178599 11.770110
11 26.451599 3.523563 11.252800
12 26.451599 2.741542 13.037108
13 26.451599 1.206997 11.644872
14 26.451599 5.145630 12.268183
15 26.451599 4.558398 11.689209
16 26.451599 2.375098 12.323958
17 26.451599 0.341757 10.999144
18 26.451599 2.519849 13.670425
19 26.451599 0.888103 12.005256
20 26.451599 1.555435 12.194905
21 26.451599 -0.366331 10.763610
22 26.451599 2.180484 12.543610
23 26.451599 -0.490246 10.928715
24 26.451599 1.026621 12.311952
25 26.451599 -1.298930 10.577192
26 26.451599 1.052756 12.253244
27 26.451599 -1.316499 10.552510
28 26.451599 1.004994 12.281128
29 26.451599 -1.019222 10.621628

测试

In [44]:
def test_loaded_model():
    # Hardcoded model file path
    filepath = f"{experiment_name}/models/MIModel_fitness92.937405_npop100_gens50_enemy[4]_mut0.20_elitism0.20_20241019_203405.txt"

    fitness_scores = []
    gains = []  # Used to store individual_gain for each run

    # Load the saved model
    solution = load_best_solution(filepath)

    # Run 5 times
    for i in range(5):
        fitness, player_life, enemy_life = simulation_test(env, solution)
        fitness_scores.append(fitness)

        # Calculate individual_gain
        individual_gain = player_life - enemy_life
        gains.append(individual_gain)

        print(f"Run {i+1} fitness: {fitness}, individual_gain: {individual_gain}")

    # Calculate the average fitness
    average_fitness = np.mean(fitness_scores)
    average_gain = np.mean(gains)

    print(f"\nAverage fitness over 5 runs: {average_fitness}")
    print(f"Average individual gain over 5 runs: {average_gain}")


# # Run test
# test_loaded_model()


def evaluate_models(experiment_name, enemy):
    # Get all model files
    model_files = glob.glob(f"{experiment_name}/*.txt")

    # Filter model files for the same enemy
    enemy_model_files = [file for file in model_files if f"enemy{enemy}" in file]

    best_gain = -np.inf
    best_model = None
    best_model_gains = []

    for filepath in enemy_model_files:
        fitness_scores = []
        gains = []  # Used to store individual_gain for each run

        # Load the saved model
        solution = load_best_solution(filepath)

        # Run 5 times
        for i in range(5):
            fitness, player_life, enemy_life = simulation_test(env, solution)
            fitness_scores.append(fitness)

            # Calculate individual_gain
            individual_gain = player_life - enemy_life
            gains.append(individual_gain)

            print(
                f"Model: {filepath}, Run {i+1} fitness: {fitness}, individual_gain: {individual_gain}"
            )

        # Calculate the average fitness
        average_fitness = np.mean(fitness_scores)
        average_gain = np.mean(gains)

        print(f"\nModel: {filepath}, Average fitness over 5 runs: {average_fitness}")
        print(
            f"Model: {filepath}, Average individual gain over 5 runs: {average_gain}\n"
        )

        # Find the model with the best result
        if average_gain > best_gain:
            best_gain = average_gain
            best_model = filepath
            best_model_gains = gains

    print(f"Best model: {best_model} with average individual gain: {best_gain}")
    print(f"Best model's 5 individual gains: {best_model_gains}")

In [46]:
env.enemies=[1,2,3,4,5,6,7,8]
env.multiplemode = 'yes'
test_loaded_model()

Run 1 fitness: 12.679341189765218, individual_gain: -16.497979459441503
Run 2 fitness: 12.679341189765218, individual_gain: -16.497979459441503
Run 3 fitness: 12.679341189765218, individual_gain: -16.497979459441503
Run 4 fitness: 12.679341189765218, individual_gain: -16.497979459441503
Run 5 fitness: 12.679341189765218, individual_gain: -16.497979459441503

Average fitness over 5 runs: 12.679341189765218
Average individual gain over 5 runs: -16.497979459441503
